In [ ]:
# q1 edie
def s(n):
    'function with input n, output is the sum of all divisors of n (except n)'
    
    #list of divisors
    divisors = []

    #checks every number up to n to see if they are divisors, and if they are, add to the divisors list
    for i in range(1, (n//2) + 1):
        if (n % i) == 0:
            divisors.append(i)
            
    #returns the sum of all divisors of n, except for n itself.
    return sum(divisors)

# optimise? storing them as pairs
def s_t(n):
    'function with input n, output is the sum of all divisors of n (except n)'
    
    #list of divisors
    divisors = []
    root_n = int(sqrt(n))
    
    #checks every number up to n to see if they are divisors, and if they are, 
    #              add to the divisors list
    for i in range(1, root_n + 1):
        if (n % i) == 0:
            i_pair = n//i
            divisors.append(i)
            # to optimise list, store them in pairs to make sequence faster
            if i_pair != n:
                divisors.append(i_pair)
                 
    #returns the sum of all divisors of n, except for n itself.
    return sum(divisors)

# q2 edie

# setting up boolean outside of function to use in q4
exceed_limit = False
LIMIT = 10**9
# establishing list outside function
aliquot = []  

def a(n, iterations):
    global aliquot
    returned_list = []
    aliquot = []
    exceed_limit = False

    # Stop conditions
    if n == 0:
        aliquot.append(n)
        returned_list = aliquot
        aliquot = []
        return returned_list

    if len(aliquot) >= iterations:
        returned_list = aliquot
        aliquot = []
        return returned_list

    if n > LIMIT:
        aliquot.append(n)
        returned_list = aliquot
        aliquot = []
        exceed_limit = True
        return returned_list

    # add current term and recurse
    aliquot.append(n)
    next_num = s_t(n)
    return a(next_num, iterations)

# q3 edie 

iterations = 23
loop_detect = []

def l(n, iterations):
    """
    returns true if the aliquot sequence of n contains a loop,
    False otherwise. Does not modify or call a(n).
    """
    seen = set()
    current = n

    for _ in range(iterations):
        if current == 0:
            return False  # sequence terminates at 0, no loop

        if current in seen:
            return True  # loop detected

        seen.add(current)

        # compute next term
        current = s(current)  # your sum-of-divisors function

        if current > 10**9:
            return False  # sequence exceeds limit, stop here

    return False  # reached max iterations without loops

# q4 edie
# LOWKEY DOESNT WORK (TOO MANY ITERATIONS) !!

# setting k and i values
k = 30
i = 10**9

# empty lists to store in
loop = []
term_0 = []
exceeded_i = []
reaches_k = []


for n in range(0, 200):

    loop_ind = l(n, k)
    Aliquot_sequence_n = a(n, k)
    
    # checking if it reaches 30 values
    if len(Aliquot_sequence_n) == 30:
        reaches_k.append(n)
    
    # checking if it loops using boolean set up in l(n)
    if loop_ind == True:
        loop.append(n)
        # looping functions also terminate at 0
        reaches_k.remove(n)
    
    # checking if it terminates at 0
    for j in range(len(Aliquot_sequence_n)):
        if Aliquot_sequence_n[j] == 0:
            term_0.append(n)
            break
    
    # checking if a value exceeds 10^9 using boolean set up in a(n, k)
    if exceed_limit == True:
        exceeded_i.append(n)